In [10]:
import supervision as sv
import numpy as np
from ultralytics import YOLO
import os

# Cargar el modelo YOLO

model_path = "runs/detect/train3/weights/best.pt" 
model = YOLO(model_path)

# Definir el polígono para la zona de peligro

# polygon = np.array([
#     [936, 419],
#     [1167, 429],
#     [1177, 375],
#     [984, 360]
# ])

polygon = np.array([        
    [1170,1078],
    [1308,648],
    [1600,686],
    [1700,1076]
])

# Inicializar la zona poligonal
zone = sv.PolygonZone(polygon=polygon)

# initiate annotators
box_annotator = sv.BoxAnnotator(thickness=4)
label_annotator = sv.LabelAnnotator(text_thickness=4, text_scale=2)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.RED, thickness=3, text_thickness=3, text_scale=4, display_in_zone_count=False)


def callback(frame: np.ndarray, _) -> np.ndarray:
    # detect
    results = model(frame, imgsz=1280)[0]
    detections = sv.Detections.from_ultralytics(results)
    zone.trigger(detections=detections[detections.class_id == 0])
    
    # annotate
    labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, _, confidence, class_id, _, _ in detections]
    frame = box_annotator.annotate(scene=frame, detections=detections)
    frame = label_annotator.annotate(scene=frame, detections=detections, labels=labels)
    frame = zone_annotator.annotate(scene=frame)    
    if zone.trigger(detections=detections[detections.class_id == 0]).any():
        text_anchor = sv.Point(x=1050, y=550)
        frame = sv.draw_text(scene=frame, text="PELIGRO!",text_anchor=text_anchor,text_thickness=2, text_scale = 1.5, background_color = sv.Color.RED, text_color=sv.Color.WHITE)
    return frame

sv.process_video(source_path="data/0_safe_walkway_violation/0_te1.mp4", target_path=f"results/result_0_te1.mp4", callback=callback)

from IPython import display
display.clear_output()
